In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    BatchNormalization,
    LeakyReLU,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    GlobalAveragePooling2D
)
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
import glob


In [ ]:
# Install the Kaggle API
!pip install kaggle

# Create a Kaggle folder if it doesn't exist
!mkdir -p ~/.kaggle

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json  # Change permissions

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download ananthu017/emotion-detection-fer -p /content/ --unzip


Dataset URL: https://www.kaggle.com/datasets/ananthu017/emotion-detection-fer
License(s): CC0-1.0
 98% 64.0M/65.2M [00:00<00:00, 235MB/s]
100% 65.2M/65.2M [00:00<00:00, 220MB/s]


In [ ]:
train_dir = "/content/train/"
test_dir = "/content/test/"

CLASS_NAMES = ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths and parameters
image_shape = (224, 224)  # Resize all images to 224x224
batch_size = 32  # Batch size for loading data

# Initialize ImageDataGenerator for training/validation and test
train_valid_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shifts
    height_shift_range=0.2,  # Random vertical shifts
    shear_range=0.2,  # Random shearing
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flips
    validation_split=0.125  # 10% out of the 80% data for validation (10/80 = 0.125)
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Only rescaling for test data

# Create training data generator (70% of the total dataset)
print("Training Images:")
train_data = train_valid_datagen.flow_from_directory(
    train_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'  # Specify training subset (70% of total)
)

# Create validation data generator (10% of the total dataset)
print("Validating Images:")
valid_data = train_valid_datagen.flow_from_directory(
    train_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'  # Specify validation subset (10% of total)
)

# Create test data generator (20% of the total dataset)
print("Test Images:")
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Training Images:
Found 25124 images belonging to 7 classes.
Validating Images:
Found 3585 images belonging to 7 classes.
Test Images:
Found 7178 images belonging to 7 classes.


In [ ]:
# Extract class names from the datasets using class_indices
train_class_names = list(train_data.class_indices.keys())
valid_class_names = list(valid_data.class_indices.keys())
test_class_names = list(test_data.class_indices.keys())

# Print class names
print("Training Class Names:", train_class_names)
print("Validation Class Names:", valid_class_names)
print("Test Class Names:", test_class_names)

# Print the number of classes
print("Number of Training Classes:", len(train_class_names))
print("Number of Validation Classes:", len(valid_class_names))
print("Number of Test Classes:", len(test_class_names))

class_names = train_class_names
class_names

Training Class Names: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Validation Class Names: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Test Class Names: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Number of Training Classes: 7
Number of Validation Classes: 7
Number of Test Classes: 7


['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

# Dataset Visualization


# Modeling


In [ ]:
CONFIGURATION = {
    'BATCH_SIZE':32,
    'IM_SIZE': 224,
    'LEARNING_RATE':0.001,
    'N_EPOCHS': 25,
    'DROPOUT_RATE':0.0,
    'REGULARIZATION_RATE':0.0,
    'N_FILTERS':6,
    'KERNEL_SIZE': 3,
    'N_STRIDES': 1,
    'POOL_SIZE' : 2,
    'N_DENSE_1':100,
    'N_DENSE_2': 10,
    'NUM_CLASSES' : 7
}


# Transfer Learning with AlexNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Configuration
CONFIGURATION = {'IM_SIZE': 224}  # AlexNet originally used 227x227 images

# Define the AlexNet backbone using a third-party implementation or custom code
def build_alexnet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Convolutional Layers
    x = layers.Conv2D(96, (11, 11), strides=4, activation='relu', padding='valid')(inputs)
    x = layers.MaxPooling2D((3, 3), strides=2)(x)

    x = layers.Conv2D(256, (5, 5), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((3, 3), strides=2)(x)

    x = layers.Conv2D(384, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(384, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((3, 3), strides=2)(x)

    return inputs, x

# Build AlexNet backbone
inputs, alexnet_output = build_alexnet((CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE'], 3))

# Add global average pooling to reduce dimensionality
x = layers.GlobalAveragePooling2D()(alexnet_output)

# Add fully connected layers with dropout
x = layers.Dropout(0.2)(x)

# Output layer for multi-class classification
outputs = layers.Dense(7, activation='softmax')(x)

# Define the model
model = models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 54, 54, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 26, 26, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 26, 26, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 384)         │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 384)         │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 12, 12, 256)         │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 5, 5, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 256)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,748,999 (14.30 MB)

 Trainable params: 3,748,999 (14.30 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Callbacks for early stopping and learning rate scheduling
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lambda epoch: 1e-3 * 0.9 ** epoch)


In [ ]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',  # Correct: pass as a string
    metrics=["accuracy"]
)


In [ ]:
# Fit the model using only 30 batches per epoch
history = model.fit(
    train_data,  # Use the full DirectoryIterator
    validation_data=valid_data,  # Full validation dataset
    epochs=50,
    steps_per_epoch=30,  # Use only 30 batches per epoch
    # callbacks=[early_stopping]
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


30/30 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.1828 - loss: 2.0986 - val_accuracy: 0.1729 - val_loss: 1.8619
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.2165 - loss: 1.8436 - val_accuracy: 0.2513 - val_loss: 1.8175
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.2510 - loss: 1.8264 - val_accuracy: 0.2513 - val_loss: 1.8219
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.2439 - loss: 1.8122 - val_accuracy: 0.2513 - val_loss: 1.8169
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.2917 - loss: 1.8052 - val_accuracy: 0.2513 - val_loss: 1.8301
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.2571 - loss: 1.8394 - val_accuracy: 0.2513 - val_loss: 1.8118
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.2577 - loss: 1.8270 - val_accuracy: 0.2513 - val_loss: 1.8117
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.2246 - loss: 1.8336 - val_accuracy: 0.2513 - val_loss: 1.8113
Epo

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


30/30 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.2662 - loss: 1.7624 - val_accuracy: 0.2513 - val_loss: 1.8132
Epoch 28/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.2576 - loss: 1.8159 - val_accuracy: 0.2513 - val_loss: 1.8123
Epoch 29/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.2557 - loss: 1.7997 - val_accuracy: 0.2513 - val_loss: 1.8103
Epoch 30/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.2676 - loss: 1.7824 - val_accuracy: 0.2513 - val_loss: 1.8111
Epoch 31/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.2431 - loss: 1.8013

In [ ]:
model.save("alexnet_model.keras")

# Testing and Evaluation


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Collect all images and true labels from the validation dataset
X_test_all = []
Y_true_all = []

for X_batch, Y_batch in valid_data:  # Iterate through the dataset
    X_test_all.append(X_batch)  # Directly append the numpy arrays
    Y_true_all.append(Y_batch)  # Directly append the numpy arrays

# Concatenate all batches into single arrays
X_test_all = np.concatenate(X_test_all, axis=0)
Y_true_all = np.concatenate(Y_true_all, axis=0)

# Predict on the entire test dataset
Y_pred = model.predict(X_test_all)  # Predictions for the test set
Y_pred_classes = np.argmax(Y_pred, axis=1)  # Convert predicted probabilities to class indices

# Define class labels (if not already defined)
class_labels = CLASS_NAMES  # Replace with your class names

# Function to randomly sample images and check predictions
def plot_random_samples(X_data, Y_true, Y_pred_classes, class_labels, n=10):
    random_indices = np.random.choice(len(X_data), size=n, replace=False)  # Randomly select `n` indices
    plt.figure(figsize=(15, 15))
    for i, idx in enumerate(random_indices):
        plt.subplot(1, n, i + 1)
        plt.imshow(X_data[idx].astype("uint8"))  # Assuming images are RGB and in uint8 format
        true_label = class_labels[Y_true[idx]]  # Map integer to class name
        pred_label = class_labels[Y_pred_classes[idx]]  # Map integer to class name
        color = "green" if Y_true[idx] == Y_pred_classes[idx] else "red"
        plt.title(f"True: {true_label}\nPred: {pred_label}", color=color)
        plt.axis('off')
    plt.suptitle("Random Sample Predictions (Correct: Green, Incorrect: Red)", fontsize=16)
    plt.show()

# Choose `n` random samples and visualize
n = 5  # Number of random samples to display
plot_random_samples(X_test_all, Y_true_all, Y_pred_classes, class_labels, n=n)


In [ ]:
Y_pred = model.predict(valid_data)
score = model.evaluate(valid_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Define class names (replace these with actual class names if available)
CLASS_NAMES = CLASS_NAMES# Ensure 'class_names' is defined in your code

# Function to plot accuracy and loss curves
def plot_training_curves(history):
    # Use a dark theme
    plt.style.use('dark_background')
    plt.figure(figsize=(12, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='cyan')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='cyan')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.show()

# Call the function to plot curves
plot_training_curves(history)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize lists for true labels and predictions
Y_true = []
Y_pred_classes = []

# Loop through validation data to get true labels and predictions
for X_batch, Y_batch in valid_data:
    # True labels
    Y_true.extend(Y_batch)
    # Predictions
    Y_pred = model.predict(X_batch)
    Y_pred_classes.extend(np.argmax(Y_pred, axis=1))  # Convert probabilities to class indices

# Convert lists to NumPy arrays
Y_true = np.array(Y_true)
Y_pred_classes = np.array(Y_pred_classes)


# Define class labels (ensure they match your training labels)
class_labels = CLASS_NAMES  # Replace CLASS_NAMES with your actual class names

# Generate and print the classification report
report = classification_report(Y_true, Y_pred_classes, target_names=class_labels)
print("Classification Report:\n", report)


# Calculate the confusion matrix
conf_matrix = confusion_matrix(Y_true, Y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


# Calculate normalized confusion matrix
conf_matrix_normalized = confusion_matrix(Y_true, Y_pred_classes, normalize='true')

# Plot the normalized confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Normalized Confusion Matrix')
plt.show()
